In [0]:
import tensorflow as tf

In [2]:
tf.__version__

'2.2.0-rc2'

In [0]:
import tensorflow_datasets as tfds #from here movie review dataset can be gained
import tensorflow_hub as hub

In [4]:
tfds.list_builders() #shows availbale datasets

['abstract_reasoning',
 'aeslc',
 'aflw2k3d',
 'amazon_us_reviews',
 'arc',
 'bair_robot_pushing_small',
 'beans',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'cos_e',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'diabetic_retinopathy_detection',
 'div2k',
 'dmlab',
 'downsampled_imagenet',
 'dsprites',
 'dtd',
 'duke_ultrasound',
 'dummy_dataset_shared_generator',
 'dummy_mnist',
 'emnist',
 'eraser_multi_rc',
 'esnli',
 'eurosat',
 'fashion_mnist',
 'flic',
 'flores',
 'food101',
 'gap',
 'gigaword',
 'glue',
 'gr

In [5]:
#loading data
tfds.load(name='imdb_reviews')

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTQW0F1/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTQW0F1/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTQW0F1/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


{'test': <DatasetV1Adapter shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>,
 'train': <DatasetV1Adapter shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>,
 'unsupervised': <DatasetV1Adapter shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>}

In [0]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [12]:
#see first 10 samples
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [13]:
#levels of this 10 reviews (1 means good review)
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [0]:
#converting the reviews to word to vectors with transfer learning technique
#trandfer learning : use pretrained neural network for word to vec(embedding)

pretrained_model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(pretrained_model, input_shape=[], dtype=tf.string, trainable=True) #first layer that converts text to embedding vectors

In [15]:
hub_layer(train_examples_batch[:2]) #lets pass 2 movie reviews to the model 
                                    #provides vectors for two reviews

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ]],
      dtype=float32)>

In [16]:
#now creating actual neural network for our problem
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16,activation="relu")) # 1 hidden layer
model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
#compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [18]:
#train the model
model.fit(train_data.shuffle(10000).batch(512),
         epochs=20,
         validation_data=validation_data.batch(512),
         verbose=1)                                        #20 iterations #verbose -> prints output for each iteration

Epoch 1/20
30/30 [==============================] - 3s 113ms/step - loss: 0.7710 - accuracy: 0.5219 - val_loss: 0.6984 - val_accuracy: 0.5773
Epoch 2/20
30/30 [==============================] - 3s 104ms/step - loss: 0.6622 - accuracy: 0.6124 - val_loss: 0.6297 - val_accuracy: 0.6484
Epoch 3/20
30/30 [==============================] - 3s 110ms/step - loss: 0.6016 - accuracy: 0.6785 - val_loss: 0.5816 - val_accuracy: 0.6919
Epoch 4/20
30/30 [==============================] - 3s 103ms/step - loss: 0.5535 - accuracy: 0.7183 - val_loss: 0.5433 - val_accuracy: 0.7288
Epoch 5/20
30/30 [==============================] - 3s 106ms/step - loss: 0.5104 - accuracy: 0.7531 - val_loss: 0.5026 - val_accuracy: 0.7594
Epoch 6/20
30/30 [==============================] - 3s 106ms/step - loss: 0.4657 - accuracy: 0.7881 - val_loss: 0.4683 - val_accuracy: 0.7858
Epoch 7/20
30/30 [==============================] - 3s 104ms/step - loss: 0.4251 - accuracy: 0.8111 - val_loss: 0.4385 - val_accuracy: 0.8031
Epoch 

In [19]:
model.predict(["This is the worst movie I have ever seen",
              "An excellent movie that I enjoyed a lot",
              "how can one make such a horrible movie? there is no story, acting direction everything is very poor"])

array([[0.07370128],
       [0.99929893],
       [0.01090534]], dtype=float32)

In [20]:
model.predict(["An excellent movie that I enjoyed a lot"])

array([[0.99929893]], dtype=float32)